# pip install xgboost

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor,IsolationForest
from sklearn.preprocessing import RobustScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

properati = pd.read_csv('datos/properati_final_y_descripcion.csv')
zonas = pd.read_csv('datos/todos_los_barrios.csv')

<IPython.core.display.Javascript object>

In [11]:
#paso features con strings a numericos
le_zona = preprocessing.LabelEncoder()
properati['state_name'] =le_zona.fit_transform(properati['state_name'])

le_barrio = preprocessing.LabelEncoder()
le_barrio.fit(zonas.values)
properati['place_name'] = le_barrio.transform(properati['place_name'])

le_tipo = preprocessing.LabelEncoder()
properati['property_type'] = le_tipo.fit_transform(properati['property_type'])

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Regresor

In [12]:
#preparo set de datos
originales=["surface_total_in_m2","surface_covered_in_m2","place_name","property_type"]
features_nuevas=["cochera","comedor","living","amplio","suite","lavadero","terraza","hall","placard","balcon","parrilla","aire","patio","muebles","pileta"\
                ,"mesada","parque","quincho","escalera","chalet","monoambiente","porcelanato","caldera","solarium","duplex","cancha"]
columnas=originales+features_nuevas
X= properati.loc[:,columnas]    
y = properati["price_aprox_usd"]

In [13]:
from xgboost import XGBRegressor

In [14]:
%%notify

xgb= XGBRegressor(nthread=-1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

learning_rate=np.arange(0.1,0.8,0.2)

param_grid = {"learning_rate": learning_rate,'min_child_weight':[1,3,5,6],'gamma':[i/10.0 for i in range(3,6)],'subsample':[i/10.0 for i in range(6,11)]}

search = RandomizedSearchCV(xgb, param_distributions=param_grid ,cv=5,n_iter=10) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 142.76 segundos para 10 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.843 (std: 0.011)
Promedio validation score: 0.610 (std: 0.120)
Promedio fit time: 2.544s
Hiper-parametros: {'subsample': 0.8, 'learning_rate': 0.50000000000000011, 'gamma': 0.5, 'min_child_weight': 3}

Puesto: 2
Promedio training score: 0.840 (std: 0.009)
Promedio validation score: 0.608 (std: 0.106)
Promedio fit time: 2.071s
Hiper-parametros: {'subsample': 1.0, 'learning_rate': 0.50000000000000011, 'gamma': 0.3, 'min_child_weight': 5}

Puesto: 3
Promedio training score: 0.853 (std: 0.010)
Promedio validation score: 0.558 (std: 0.123)
Promedio fit time: 1.978s
Hiper-parametros: {'subsample': 1.0, 'learning_rate': 0.70000000000000007, 'gamma': 0.3, 'min_child_weight': 6}

Puesto: 4
Promedio training score: 0.857 (std: 0.012)
Promedio validation score: 0.557 (std: 0.156)
Promedio fit time: 2.179s
Hiper-parametros: {'subsample': 1.0, 'learning_rate': 0.50000000000000011, 'gamma':

<IPython.core.display.Javascript object>

# Clasificador

In [1]:
from xgboost import XGBClassifier

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
%%notify

xgb= XGBClassifier(nthread=-1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

learning_rate=np.arange(0.1,0.8,0.2)

param_grid = {"learning_rate": learning_rate,'min_child_weight':[1,3,5,6],'gamma':[i/10.0 for i in range(3,6)],'subsample':[i/10.0 for i in range(6,11)]}

search = RandomizedSearchCV(xgb, param_distributions=param_grid ,cv=5,n_iter=10) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

###  Analsis de errores

In [15]:
search.score(X_test,y_test)

0.66754097761115272

In [17]:
mejor = search.best_estimator_

In [24]:
errores = mejor.predict(X_test)-y_test
errores_porc = 100*((mejor.predict(X_test)-y_test))/y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print ("Con % \nError maximo:{0}%\nError minimo:{1}%".format( max(abs(errores_porc)),min(abs(errores_porc))))
print(errores)

Error maximo:7834287.5
Error minimo:0.34375
Con % 
Error maximo:8607.34%
Error minimo:6.61057692308e-05%
30779    -129160.812500
28531       4183.015625
106253     26671.484375
26654      53804.359375
46425     -70954.843750
8200       33493.625000
62512     462750.937500
11033      38003.023438
16582     171109.250000
17941     -62596.562500
35352      -9361.039062
17319      95083.750000
98717    -316642.937500
73811       9960.828125
84166      61521.296875
109154      6049.468750
122463    -73068.093750
6748     -248170.875000
77650      68061.812500
113977    -34287.390625
118295      6006.171875
19991     453355.375000
11371     -21604.796875
50446       5177.636719
55404     -26239.500000
4028       49375.296875
48481       1997.335938
73287     -24779.656250
11449     -39766.703125
106707     19670.328125
              ...      
52067     -78178.851562
108684     43815.968750
55125      33063.781250
130089     -3163.750000
125020     -9094.765625
99295      41242.375000
21972  

In [25]:
count_max=0
max_error=100
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 1233 de 27271 que tienen mas de 100 % de error (4%)


In [26]:
count_max=0
max_error=20
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 8558 de 27271 que tienen mas de 20 % de error (31%)


In [27]:
count_max=0
max_error=10
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 11428 de 27271 que tienen mas de 10 % de error (41%)


In [28]:
count_max=0
max_error=5
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 13040 de 27271 que tienen mas de 5 % de error (47%)
